In [1]:
import os 
import csv
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
cache_misses = pd.read_csv("../ce_data/caches/llc_miss_ratio.csv")

In [3]:
rnd = [random.randrange(189), random.randrange(189), random.randrange(189)]
print(rnd)

[31, 105, 128]


In [4]:
reuse_distances = pd.DataFrame()
test_distances = pd.DataFrame()
tmp_ctr = 0
for file in os.listdir("../ce_data/reuse_distances"):
    x = pd.read_csv("../ce_data/reuse_distances/"+file)
    if tmp_ctr in rnd:
        test_distances = pd.concat([test_distances, x], ignore_index=True)
    else:
        reuse_distances = pd.concat([reuse_distances, x], ignore_index=True)
    tmp_ctr +=1

In [5]:
test_misses = cache_misses.loc[cache_misses['id'].isin(test_distances['id'])]
cache_misses = cache_misses.loc[~cache_misses['id'].isin(test_distances['id'])]

In [6]:
cache_misses = cache_misses.sort_values("id", ascending=True, ignore_index=True)
reuse_distances = reuse_distances.sort_values("id", ascending=True, ignore_index=True)
test_misses = test_misses.sort_values("id", ascending=True, ignore_index=True)
test_distances = test_distances.sort_values("id", ascending=True, ignore_index=True)

In [7]:
print(cache_misses)
print(reuse_distances)
print(test_distances)
print(test_misses)

                           id        32        64       128       256  \
0         400.perlbench-41B_0  1.000000  1.000000  1.000000  0.998652   
1         400.perlbench-41B_1  1.000000  1.000000  1.000000  1.000000   
2        400.perlbench-41B_10  1.000000  0.997375  0.997375  0.997375   
3       400.perlbench-41B_100  0.996656  0.996656  0.996656  0.993311   
4       400.perlbench-41B_101  1.000000  1.000000  1.000000  1.000000   
...                       ...       ...       ...       ...       ...   
139309        657.xz_s-56B_95  1.000000  1.000000  1.000000  1.000000   
139310        657.xz_s-56B_96  1.000000  1.000000  1.000000  1.000000   
139311        657.xz_s-56B_97  1.000000  1.000000  1.000000  1.000000   
139312        657.xz_s-56B_98  1.000000  1.000000  1.000000  1.000000   
139313        657.xz_s-56B_99  1.000000  1.000000  1.000000  1.000000   

             512      1024      2048      4096      8192  
0       0.889488  0.828841  0.803235  0.773585  0.768194  
1    

In [8]:
# Quick check if all columns are the same
for i,j in zip(reuse_distances['id'], cache_misses['id']):
    if i!=j:
        print(i, j)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(reuse_distances,cache_misses, test_size=0.1)

In [10]:
X_train_ids = X_train["id"]
X_test_ids = X_test["id"]
y_train_ids = y_train["id"]
y_test_ids = y_test["id"]

print(X_train_ids)

31748          445.gobmk-36B_36
42892         458.sjeng-31B_278
137200       657.xz_s-3167B_218
63839     482.sphinx3-1297B_255
12427           429.mcf-22B_498
                  ...          
18581      435.gromacs-226B_643
18363      435.gromacs-226B_447
57268        473.astar-359B_408
130968     654.roms_s-1021B_677
5781            403.gcc-17B_583
Name: id, Length: 125382, dtype: object


In [11]:
X_train = X_train.drop(columns=["id"])
X_test = X_test.drop(columns=["id"])
y_train = y_train.drop(columns=["id"])
y_test = y_test.drop(columns=["id"])

In [12]:
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(125382, 896) (13932, 896)
(125382, 9) (13932, 9)


In [13]:
print(X_train.columns)

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '3623878656', '3690987520', '3758096384', '3825205248', '3892314112',
       '3959422976', '4026531840', '4093640704', '4160749568', '4227858432'],
      dtype='object', length=896)


In [14]:
from keras.models import Sequential
from keras.layers import Dense
from keras.metrics import MeanSquaredError
from keras.losses import MeanSquaredError
# Create the model
model = Sequential()

# Add the input layer and the first hidden layer
model.add(Dense(512, input_dim=896, activation='sigmoid'))

# Add the second hidden layer
model.add(Dense(512, activation='sigmoid'))

# Add the output layer
model.add(Dense(9, activation='sigmoid'))

# Compile the model
model.compile(loss=MeanSquaredError() , optimizer='adam')

# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               459264    
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 dense_2 (Dense)             (None, 9)                 4617      
                                                                 
Total params: 726,537
Trainable params: 726,537
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(X_train.to_numpy(), y_train.to_numpy(), epochs=20)

Epoch 1/20
3919/3919 [==============================] - 23s 6ms/step - loss: 0.0695
Epoch 2/20
3919/3919 [==============================] - 22s 6ms/step - loss: 0.0675
Epoch 3/20
3919/3919 [==============================] - 22s 6ms/step - loss: 0.0686
Epoch 4/20
3919/3919 [==============================] - 22s 6ms/step - loss: 0.0646
Epoch 5/20
3919/3919 [==============================] - 22s 6ms/step - loss: 0.0626
Epoch 6/20
3919/3919 [==============================] - 22s 6ms/step - loss: 0.0593
Epoch 7/20
3919/3919 [==============================] - 22s 6ms/step - loss: 0.0609
Epoch 8/20
3919/3919 [==============================] - 22s 6ms/step - loss: 0.0543
Epoch 9/20
3919/3919 [==============================] - 22s 6ms/step - loss: 0.0553
Epoch 10/20
3919/3919 [==============================] - 22s 6ms/step - loss: 0.0578
Epoch 11/20
3919/3919 [==============================] - 22s 6ms/step - loss: 0.0545
Epoch 12/20
3919/3919 [==============================] - 22s 6ms/step - lo

In [16]:
f = model(X_test.to_numpy())

In [17]:
from sklearn.metrics import mean_squared_error
# Compare columns
for i,j in enumerate(['32','64','128','256','512','1024','2048','4096','8192']):
    model_out = f.numpy()[:,i]
    y_out = y_test.to_numpy()[:,i]
    print(f'Mean squared error for cache size {j}\t: {mean_squared_error(model_out, y_out)}')

Mean squared error for cache size 32	: 0.04380860296204889
Mean squared error for cache size 64	: 0.046249517710557936
Mean squared error for cache size 128	: 0.049495517588676895
Mean squared error for cache size 256	: 0.054294298525031474
Mean squared error for cache size 512	: 0.06067280362603706
Mean squared error for cache size 1024	: 0.055309304246600494
Mean squared error for cache size 2048	: 0.06063767239948323
Mean squared error for cache size 4096	: 0.06147204758753704
Mean squared error for cache size 8192	: 0.06127600581091548


In [18]:
print(f)

tf.Tensor(
[[0.9995916  0.9992937  0.9985631  ... 0.8802107  0.8664787  0.7873012 ]
 [0.99764    0.99599594 0.99151886 ... 0.4698933  0.3877806  0.31591353]
 [0.99871844 0.99768704 0.9958874  ... 0.59711546 0.4763542  0.4271807 ]
 ...
 [0.9995992  0.9992164  0.99837434 ... 0.905261   0.91674143 0.8939051 ]
 [0.99847585 0.99693114 0.9920211  ... 0.39229855 0.33465594 0.3151484 ]
 [0.9991931  0.998745   0.9980849  ... 0.91245985 0.85329366 0.808674  ]], shape=(13932, 9), dtype=float32)


In [19]:
print(y_test.to_numpy(), 'shape =', y_test.to_numpy().shape)

[[1.         1.         1.         ... 1.         1.         1.        ]
 [0.9880035  0.97784483 0.96508214 ... 0.15846044 0.03031499 0.        ]
 [0.99947674 0.9988862  0.99739864 ... 0.56432957 0.48378658 0.43592791]
 ...
 [1.         1.         1.         ... 0.97572118 0.97572118 0.97572118]
 [0.99929575 0.9983978  0.99698944 ... 0.64754762 0.34766533 0.08350205]
 [1.         1.         1.         ... 1.         1.         1.        ]] shape = (13932, 9)


In [20]:
print(test_distances)

                         id      0      1      2      3       4      5      6  \
0       437.leslie3d-271B_0  17824  30528  12064  24016   37696  44832  50080   
1       437.leslie3d-271B_1  17712  30544  11840  24096   37280  45728  51296   
2      437.leslie3d-271B_10  21936  32016  30080  50848  125616  46768  21888   
3     437.leslie3d-271B_100   9344  18224  31936  35248   37088  28992  23216   
4     437.leslie3d-271B_101   9040  17872  33328  35280   34432  28496  24448   
...                     ...    ...    ...    ...    ...     ...    ...    ...   
2242     621.wrf_s-6673B_95  12160  27968  31488  35040   36176  58208  32784   
2243     621.wrf_s-6673B_96  12784  29552  30416  33456   38048  62448  34768   
2244     621.wrf_s-6673B_97  13808  23216  25904  27520   33152  47568  38912   
2245     621.wrf_s-6673B_98  13808  18912  20272  22368   23312  29456  43408   
2246     621.wrf_s-6673B_99  14992  17728  19504  23120   24240  26960  33440   

          7      8  ...  36

In [21]:
tm_ids = test_misses["id"]
test_misses = test_misses.drop(columns=["id"])

In [22]:
td_ids = test_distances["id"]
test_distances = test_distances.drop(columns=["id"])

In [24]:
f = model(test_distances.to_numpy())

In [25]:
from sklearn.metrics import mean_squared_error
# Compare columns
for i,j in enumerate(['32','64','128','256','512','1024','2048','4096','8192']):
    model_out = f.numpy()[:,i]
    y_out = test_misses.to_numpy()[:,i]
    print(f'Mean squared error for cache size {j}\t: {mean_squared_error(model_out, y_out)}')

Mean squared error for cache size 32	: 0.0014185103344748486
Mean squared error for cache size 64	: 0.0037782744835771916
Mean squared error for cache size 128	: 0.009222021334196692
Mean squared error for cache size 256	: 0.022473753329407913
Mean squared error for cache size 512	: 0.06390969910777379
Mean squared error for cache size 1024	: 0.04887628637288899
Mean squared error for cache size 2048	: 0.06957508706674734
Mean squared error for cache size 4096	: 0.06996495192355792
Mean squared error for cache size 8192	: 0.07727743426022149
